# Jersey City GeoCoding

## loading the dataframes

In [8]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)
# loading saved objects into dataframe
errors_df = pd.read_pickle('errors_df.pkl')

df = pd.read_pickle('JC_residential.pkl')

In [9]:
df[df.gCode.str.contains('Jersey City') == False]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode,latitude,longitude


In [10]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode,latitude,longitude
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307","677 LIBERTY AVE. Jersey City, NJ",1,1,"(677, Liberty Avenue, Jersey City, Hudson Coun...",40.759744,-74.050457
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307","675 LIBERTY AVE. Jersey City, NJ",1,1,"(675, Liberty Avenue, Jersey City, Hudson Coun...",40.759679,-74.050499
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307","673 LIBERTY AVE. Jersey City, NJ",1,1,"(673, Liberty Avenue, Jersey City, Hudson Coun...",40.759614,-74.050542
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307","671 LIBERTY AVE. Jersey City, NJ",1,1,"(671, Liberty Avenue, Jersey City, Hudson Coun...",40.759549,-74.050584
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307","669 LIBERTY AVE. Jersey City, NJ",1,1,"(669, Liberty Avenue, Jersey City, Hudson Coun...",40.759484,-74.050626
...,...,...,...,...,...,...,...,...,...,...,...
38273,100 EAST SHEARWATER CT.,"BALANLAYOS, JESUS & MYRNA",31 HARMONY WAY,"SEWAREN, NJ 07077","31 HARMONY WAY, SEWAREN, NJ 07077","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38276,100 EAST SHEARWATER CT.,"DANGCIL, PROCERFINA & WILLIAM A JR",100-51 E SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-51 E SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38278,100 EAST SHEARWATER CT.,"LAWSONS REALTY,LLC",130 SHORE RD #198,"PORT WASHINGTON, NY 11050","130 SHORE RD #198, PORT WASHINGTON, NY 11050","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38279,100 EAST SHEARWATER CT.,"NATHAN, NAVIN D.",100-54 SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-54 SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935


In [3]:
errors_df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


## helper functions

In [9]:
#jc_deed_df = pd.read_csv('0906130957_deed.csv', index_col=False)

#df = pd.read_csv('0906monm220028.csv', usecols=['propertyLocation', 'ownersName', 'ownersMailingAddress', 'cityStateZip'])

# number of times owner appears in second column
def propertiesOwned(owner=str):
    return len(df[df.ownersName == owner])

# number of times property appears in first column
def propertyAppearances(address=str): 
    return len(df[df.propertyLocation == address])
#
def getPropertiesOwned(owner=str):
    return df[df.ownersName == owner]['propertyLocation'].unique().tolist()

#df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip
#df['propertyFullAddress'] = df.propertyLocation + ' Jersey City NJ'


In [10]:
errors_df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [2]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [4]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
2529,78A RNE ST.,"MITTAL,CHITRA KOTHARI & DEEPAK",76 CONSTITUTION WAY,"JERSEY CITY, N.J. 07302","76 CONSTITUTION WAY, JERSEY CITY, N.J. 07302","78A RNE ST. Jersey City, NJ",1,1,None
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [30]:
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TONNELE', 'TONNELLE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC ADOO', 'MCADOO')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DRIVE', 'MARTIN LUTHER KING DRIVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COLUMBUS', 'CHRISTOPHER COLUMBUS')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIRST', '1st')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SECOND', '2nd')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THIRD', '3rd')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOURTH', '4th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTH', '5th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTH', '6th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVENTH', '7th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EIGHTH', '8th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINETH', '9th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TENTH', '10th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINTH', '9th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTEENTH', '15th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTEENTH', '16th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC DOUGALL', 'MCDOUGALL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTIER', 'CARPENTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOX HOUND', 'FOXHOUND')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIEDLER', 'SEIDLER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIENAU', 'LINEAU')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('KENNEDY', 'JOHN F KENNEDY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIBERTY ST.', 'LIBERTY AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-REAR', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('(REAR)', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-FRONT', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HAMPTON CT.', 'HAMPTON COURT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VARCK', 'VARICK')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THO', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE AVE', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TER', 'TR')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3633A', '3633')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3144A', '3144')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SHEARWATR', 'SHEARWATER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PALUSL', "PAUL'S")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COURT TR', "CT")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EASTVIEW', "E VIEW")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DR.', 'MARTIN LUTHER KING DR.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 1ST FL.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' SO.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' N.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ODGEN', 'OGDEN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PATRSON', 'PATERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 61 COLES', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VAN HOUTEN ST.', 'VAN HOUTEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON ST.', 'MCPHERSON PL.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON', 'MCPHERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIR ROAD', 'FIR ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('UNION AVE.', 'UNION ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('OLD BERGEN', 'OLD BERGEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE. AVE.', 'AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('JACKSON AVE.', 'JACKSON ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('162 OLD BERGEN AVE. AVE.', '162 OLD BERGEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLEN', 'GLENN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLENNN', 'GLENN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DR.VE', 'DRIVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('1 GLENNN LANE', '1 GLENN LANE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\(\)', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}(.?\d{1}(A)?)?A?\b', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\s#\d', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BUTTRNUT', 'BUTTERNUT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTR', 'CARPENTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('100 SHEARWATER CT.', '100 EAST SHEARWATER CT.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('78A RNE ST.', '78 THORNE ST.')

# overwriting past mistakes
errors_df.propertyFullAddress = errors_df.propertyLocation + ' Jersey City, NJ'

/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1440/1358142991.py:69: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = errors_df.propertyLocation.str.replace('78A RNE ST.', '78 THORNE ST.')


In [31]:
%%time
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

CPU times: user 21.6 ms, sys: 3.5 ms, total: 25.1 ms
Wall time: 2.91 s


In [32]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [33]:
errors_df.iloc[geocode_errors].propertyLocation.unique()

array(['10 PAULMIER PL.', '280 GREGORY PARK PLAZA',
       '270 GREGORY PARK PLAZA', '32 WILKINSON ST.', '107 SHEARWATER CT.'],
      dtype=object)

In [34]:
len(errors_df.iloc[geocode_errors].propertyLocation.unique())

5

### Before we save the smaller errors data frame, we need to append the cleaned errors to the original df

In [35]:
df.append(errors_df.dropna(axis=0, subset=['gCode'])).to_pickle('JC_residential.pkl')

In [36]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [37]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [38]:
# reassigning the errors_df to the remaining errors
errors_df = errors_df.iloc[geocode_errors].copy()

In [39]:
errors_df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode
19294,10 PAULMIER PL.,"DOWLING, JAMES PATRICK & TOBY K.",10 PAULMIER PL,"JERSEY CITY, N J 07302","10 PAULMIER PL, JERSEY CITY, N J 07302","10 PAULMIER PL. Jersey City, NJ",1,1,None
19691,280 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,280 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","280 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
19692,270 GREGORY PARK PLAZA,METROPOLIS TOWERS APT. CORP. MGT.,270 GREGORY PARK PLAZA,"JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA, JERSEY CITY, NJ 07302","270 GREGORY PARK PLAZA Jersey City, NJ",2,1,None
32583,32 WILKINSON ST.,"HARIKRISHAN, HARRY",23 TERHUNE AVE.,"JERSEY CITY, NJ 07305","23 TERHUNE AVE., JERSEY CITY, NJ 07305","32 WILKINSON ST. Jersey City, NJ",1,1,None
38254,107 SHEARWATER CT.,"SANTANIELLO III, PHILLIP & CLAIRE",107 SHEARWATER CT.,"JERSEY CITY, NJ 07305","107 SHEARWATER CT., JERSEY CITY, NJ 07305","107 SHEARWATER CT. Jersey City, NJ",1,1,None


In [40]:
errors_df.to_pickle('errors_df.pkl')

In [41]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode,latitude,longitude
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307","677 LIBERTY AVE. Jersey City, NJ",1,1,"(677, Liberty Avenue, Jersey City, Hudson Coun...",40.759744,-74.050457
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307","675 LIBERTY AVE. Jersey City, NJ",1,1,"(675, Liberty Avenue, Jersey City, Hudson Coun...",40.759679,-74.050499
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307","673 LIBERTY AVE. Jersey City, NJ",1,1,"(673, Liberty Avenue, Jersey City, Hudson Coun...",40.759614,-74.050542
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307","671 LIBERTY AVE. Jersey City, NJ",1,1,"(671, Liberty Avenue, Jersey City, Hudson Coun...",40.759549,-74.050584
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307","669 LIBERTY AVE. Jersey City, NJ",1,1,"(669, Liberty Avenue, Jersey City, Hudson Coun...",40.759484,-74.050626
...,...,...,...,...,...,...,...,...,...,...,...
38273,100 EAST SHEARWATER CT.,"BALANLAYOS, JESUS & MYRNA",31 HARMONY WAY,"SEWAREN, NJ 07077","31 HARMONY WAY, SEWAREN, NJ 07077","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38276,100 EAST SHEARWATER CT.,"DANGCIL, PROCERFINA & WILLIAM A JR",100-51 E SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-51 E SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38278,100 EAST SHEARWATER CT.,"LAWSONS REALTY,LLC",130 SHORE RD #198,"PORT WASHINGTON, NY 11050","130 SHORE RD #198, PORT WASHINGTON, NY 11050","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38279,100 EAST SHEARWATER CT.,"NATHAN, NAVIN D.",100-54 SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-54 SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935


In [42]:
df['latitude'] = [g.latitude for g in df.gCode]
df['longitude'] = [g.longitude for g in df.gCode]

### Check to see if any of the returned geocodes contains a city other than 'JERSEY CITY'

In [43]:
df.append(errors_df.dropna(axis=0, subset=['gCode'])).to_pickle('JC_residential.pkl')

In [44]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,propertiesOwned,units,gCode,latitude,longitude
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307","677 LIBERTY AVE. Jersey City, NJ",1,1,"(677, Liberty Avenue, Jersey City, Hudson Coun...",40.759744,-74.050457
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307","675 LIBERTY AVE. Jersey City, NJ",1,1,"(675, Liberty Avenue, Jersey City, Hudson Coun...",40.759679,-74.050499
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307","673 LIBERTY AVE. Jersey City, NJ",1,1,"(673, Liberty Avenue, Jersey City, Hudson Coun...",40.759614,-74.050542
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307","671 LIBERTY AVE. Jersey City, NJ",1,1,"(671, Liberty Avenue, Jersey City, Hudson Coun...",40.759549,-74.050584
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307","669 LIBERTY AVE. Jersey City, NJ",1,1,"(669, Liberty Avenue, Jersey City, Hudson Coun...",40.759484,-74.050626
...,...,...,...,...,...,...,...,...,...,...,...
38273,100 EAST SHEARWATER CT.,"BALANLAYOS, JESUS & MYRNA",31 HARMONY WAY,"SEWAREN, NJ 07077","31 HARMONY WAY, SEWAREN, NJ 07077","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38276,100 EAST SHEARWATER CT.,"DANGCIL, PROCERFINA & WILLIAM A JR",100-51 E SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-51 E SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38278,100 EAST SHEARWATER CT.,"LAWSONS REALTY,LLC",130 SHORE RD #198,"PORT WASHINGTON, NY 11050","130 SHORE RD #198, PORT WASHINGTON, NY 11050","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935
38279,100 EAST SHEARWATER CT.,"NATHAN, NAVIN D.",100-54 SHEARWATER CT.,"JERSEY CITY, NJ 07305","100-54 SHEARWATER CT., JERSEY CITY, NJ 07305","100 EAST SHEARWATER CT. Jersey City, NJ",1,1,"(100, East Shearwater Court, Jersey City, Huds...",40.686674,-74.071935


In [19]:
df.to_csv('JC_residential.csv')

In [60]:
# return the dataframe for those gCodes which don't contain 'Hudson' or 'Essex'
# append a copy of this dataframe to errors_df
df[df.gCode.str.contains('Hudson|Essex') == False]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,numberPropertiesOwned,units,gCode,Lat,Long,propertiesOwned
11035,10 BERKLEY PL.,"PATEL INDO PAK GROCERY, INC.",52 ROBIN HOOD ROAD,"CLIFTON, NJ 07013","52 ROBIN HOOD ROAD, CLIFTON, NJ 07013",10 BERKLEY PL. Jersey City NJ,2,1,"10, Berkley Place, Dumont, Bergen County, New ...",40.949309,-73.998756,"[10 BERKLEY PL., 12 BERKLEY PL.]"
11036,12 BERKLEY PL.,"PATEL INDO PAK GROCERY, INC.",52 ROBINHOOD RD.,"CLIFTON, NJ 07013","52 ROBINHOOD RD., CLIFTON, NJ 07013",12 BERKLEY PL. Jersey City NJ,2,1,"12, Berkley Place, Dumont, Bergen County, New ...",40.949357,-73.998758,"[10 BERKLEY PL., 12 BERKLEY PL.]"
11150,19 BERKLEY PL.,"DREWA, WIKTOR & DANIELA",17 BERKLEY PL.,"JERSEY CITY, N.J. 07306","17 BERKLEY PL., JERSEY CITY, N.J. 07306",19 BERKLEY PL. Jersey City NJ,1,1,"19, Berkley Place, East Windsor Township, Merc...",40.250864,-74.533198,[19 BERKLEY PL.]
11151,17 BERKLEY PL.,"DREWA, DANIELA",17 BERKLEY PL.,"JERSEY CITY, N.J. 07306","17 BERKLEY PL., JERSEY CITY, N.J. 07306",17 BERKLEY PL. Jersey City NJ,1,1,"17, Berkley Place, East Windsor Township, Merc...",40.250656,-74.533229,[17 BERKLEY PL.]
11152,15 BERKLEY PL.,"KULKARNI, JITENDRA M. & DEEPA J.",27 FOREST ROAD,"NEW PROVIDENCE, NJ 07974","27 FOREST ROAD, NEW PROVIDENCE, NJ 07974",15 BERKLEY PL. Jersey City NJ,1,1,"15, Berkley Place, East Windsor Township, Merc...",40.250415,-74.533237,[15 BERKLEY PL.]
...,...,...,...,...,...,...,...,...,...,...,...,...
40921,6 HAMPTON CT.,"ST KILDA, LLC. C/O US PROPERTY FUND","200 HUDSON ST., STE 1000","JERSEY CITY, NJ 07311","200 HUDSON ST., STE 1000, JERSEY CITY, NJ 07311",6 HAMPTON CT. Jersey City NJ,2,1,"6, Hampton Court, Chestnut Glen, Davisville, G...",39.799217,-75.049115,"[346 ARMSTRONG AVE., 6 HAMPTON CT.]"
40922,5 HAMPTON CT.,"MELBOURNE, LLC.","140 BROADWAY, 28TH FL.","NEW YORK, NY 10005","140 BROADWAY, 28TH FL., NEW YORK, NY 10005",5 HAMPTON CT. Jersey City NJ,18,1,"5, Hampton Court, Chestnut Glen, Davisville, G...",39.799247,-75.050097,"[229 BOWERS ST., 40 MANHATTAN AVE., 132 PROSPE..."
40923,4 HAMPTON CT.,"DAVISON , JOYCE",4 HAMPTON CT. TERR.,"JERSEY CITY, N.J. 07302","4 HAMPTON CT. TERR., JERSEY CITY, N.J. 07302",4 HAMPTON CT. Jersey City NJ,1,1,"4, Hampton Court, Chestnut Glen, Davisville, G...",39.799012,-75.049027,[4 HAMPTON CT.]
40924,3 HAMPTON CT.,"BUCK,ZACHARY C. & DIANA Y.PERALES",30 S. KINGMAN RD,"SOUTH ORANGE, NJ 07079","30 S. KINGMAN RD, SOUTH ORANGE, NJ 07079",3 HAMPTON CT. Jersey City NJ,1,1,"3, Hampton Court, Chestnut Glen, Davisville, G...",39.799139,-75.049806,[3 HAMPTON CT.]


In [62]:
errors_df = df[df.gCode.str.contains('Hudson|Essex') == False].copy()

### Cleaning Loop

In [63]:
errors_df.propertyLocation.unique()

array(['10 BERKLEY PL.', '12 BERKLEY PL.', '19 BERKLEY PL.',
       '17 BERKLEY PL.', '15 BERKLEY PL.', '13 BERKLEY PL.',
       '11 BERKLEY PL.', '9 BERKLEY PL.', '12 LOTT STREET', '14 LOTT ST.',
       '16 LOTT ST.', '347 NINTH ST.', '343 NINTH ST.',
       '327.5 NINTH STREET', '254.5 NINTH ST.', '260.5 NINTH STREET',
       '288.5 NINTH STREET', '322 NINTH ST.', '214 NINTH ST.',
       '216 NINTH ST.', '25 FAYETTE ST.', '376 FIFTH ST.',
       '376.5 FIFTH ST.', '378 FIFTH ST.', '386.5 FIFTH STREET',
       '377 FIFTH ST.', '355.5 FOURTH ST.', '347 FOURTH ST.',
       '343 FOURTH ST.', '354.5 FOURTH STREET', '344 FOURTH ST.',
       '357.5 FIFTH STREET', '353.5 FIFTH STREET', '347.5 FIFTH STREET',
       '347 FIFTH ST.', '343 FIFTH ST.', '356.5 FIFTH STREET',
       '352.5 FIFTH STREET', '348.5 FIFTH STREET', '344 FIFTH ST.',
       '344.5 FIFTH STREET', '295 NINTH ST.', '199 NINTH ST.',
       '378 THIRD ST.', '347 THIRD ST.', '344 FIRST ST.', '396 FIRST ST.',
       '398 FIRST ST

In [114]:
replacement_dict.keys()

dict_keys(['BERKELEY', 'FIRST', 'FIFTH', 'FOURTH', 'GLEN', 'NINTH', 'SECOND', 'THIRD'])

In [120]:
errors_df.propertyLocation.replace(replacement_dict, value=None, inplace=True)

In [193]:
errors_df.gCode[11035]

Location(10, Berkeley Place, Indian Square, Croxton, Jersey City, Hudson County, New Jersey, 07306, United States, (40.73702659183674, -74.06588356122448, 0.0))

In [122]:
errors_df['propertyFullAddress'] = errors_df.propertyLocation + ', Jersey City, NJ'

In [123]:
%%time
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

CPU times: user 383 ms, sys: 54.7 ms, total: 437 ms
Wall time: 1min 59s


In [184]:
errors_df[errors_df.gCode.str.contains('Hudson') == False]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,numberPropertiesOwned,units,gCode,Lat,Long,propertiesOwned


In [127]:
errors_df['gCode'].iloc[0]

Location(10, Berkeley Place, Indian Square, Croxton, Jersey City, Hudson County, New Jersey, 07306, United States, (40.73702659183674, -74.06588356122448, 0.0))

In [200]:
errors_df.gCode[11035]

Location(10, Berkeley Place, Indian Square, Croxton, Jersey City, Hudson County, New Jersey, 07306, United States, (40.73702659183674, -74.06588356122448, 0.0))

In [204]:
errors_df = df[df.gCode.str.contains('Hudson') == False].copy()

In [207]:
errors_df.gCode.replace(combined_repl_dict, value=None, inplace=True)

In [208]:
errors_df['propertyFullAddress'] = errors_df.propertyLocation + ', Jersey City, NJ'

In [209]:
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [2]:
df = pd.read_pickle('jersey_city.pkl')

In [3]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,numberPropertiesOwned,units,gCode,Lat,Long,propertiesOwned
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307",677 LIBERTY AVE. Jersey City NJ,1,1,"677, Liberty Avenue, Jersey City, Hudson Count...",40.759744,-74.050457,[677 LIBERTY AVE.]
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307",675 LIBERTY AVE. Jersey City NJ,1,1,"675, Liberty Avenue, Jersey City, Hudson Count...",40.759679,-74.050499,[675 LIBERTY AVE.]
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307",673 LIBERTY AVE. Jersey City NJ,1,1,"673, Liberty Avenue, Jersey City, Hudson Count...",40.759614,-74.050542,[673 LIBERTY AVE.]
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307",671 LIBERTY AVE. Jersey City NJ,1,1,"671, Liberty Avenue, Jersey City, Hudson Count...",40.759549,-74.050584,[671 LIBERTY AVE.]
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307",669 LIBERTY AVE. Jersey City NJ,1,1,"669, Liberty Avenue, Jersey City, Hudson Count...",40.759484,-74.050626,[669 LIBERTY AVE.]
...,...,...,...,...,...,...,...,...,...,...,...,...
40921,6 HAMPTON CT.,"ST KILDA, LLC. C/O US PROPERTY FUND","200 HUDSON ST., STE 1000","JERSEY CITY, NJ 07311","200 HUDSON ST., STE 1000, JERSEY CITY, NJ 07311","6 HAMPTON CT., Jersey City, NJ",2,1,"(6, Hampton Court, Chestnut Glen, Davisville, ...",39.799217,-75.049115,"[346 ARMSTRONG AVE., 6 HAMPTON CT.]"
40922,5 HAMPTON CT.,"MELBOURNE, LLC.","140 BROADWAY, 28TH FL.","NEW YORK, NY 10005","140 BROADWAY, 28TH FL., NEW YORK, NY 10005","5 HAMPTON CT., Jersey City, NJ",18,1,"(5, Hampton Court, Chestnut Glen, Davisville, ...",39.799247,-75.050097,"[229 BOWERS ST., 40 MANHATTAN AVE., 132 PROSPE..."
40923,4 HAMPTON CT.,"DAVISON , JOYCE",4 HAMPTON CT. TERR.,"JERSEY CITY, N.J. 07302","4 HAMPTON CT. TERR., JERSEY CITY, N.J. 07302","4 HAMPTON CT., Jersey City, NJ",1,1,"(4, Hampton Court, Chestnut Glen, Davisville, ...",39.799012,-75.049027,[4 HAMPTON CT.]
40924,3 HAMPTON CT.,"BUCK,ZACHARY C. & DIANA Y.PERALES",30 S. KINGMAN RD,"SOUTH ORANGE, NJ 07079","30 S. KINGMAN RD, SOUTH ORANGE, NJ 07079","3 HAMPTON CT., Jersey City, NJ",1,1,"(3, Hampton Court, Chestnut Glen, Davisville, ...",39.799139,-75.049806,[3 HAMPTON CT.]


In [145]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,numberPropertiesOwned,units,gCode,Lat,Long,propertiesOwned
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307",677 LIBERTY AVE. Jersey City NJ,1,1,"677, Liberty Avenue, Jersey City, Hudson Count...",40.759744,-74.050457,[677 LIBERTY AVE.]
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307",675 LIBERTY AVE. Jersey City NJ,1,1,"675, Liberty Avenue, Jersey City, Hudson Count...",40.759679,-74.050499,[675 LIBERTY AVE.]
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307",673 LIBERTY AVE. Jersey City NJ,1,1,"673, Liberty Avenue, Jersey City, Hudson Count...",40.759614,-74.050542,[673 LIBERTY AVE.]
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307",671 LIBERTY AVE. Jersey City NJ,1,1,"671, Liberty Avenue, Jersey City, Hudson Count...",40.759549,-74.050584,[671 LIBERTY AVE.]
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307",669 LIBERTY AVE. Jersey City NJ,1,1,"669, Liberty Avenue, Jersey City, Hudson Count...",40.759484,-74.050626,[669 LIBERTY AVE.]
...,...,...,...,...,...,...,...,...,...,...,...,...
40921,6 HAMPTON CT.,"ST KILDA, LLC. C/O US PROPERTY FUND","200 HUDSON ST., STE 1000","JERSEY CITY, NJ 07311","200 HUDSON ST., STE 1000, JERSEY CITY, NJ 07311",6 HAMPTON CT. Jersey City NJ,2,1,"(6, Hampton Court, Chestnut Glen, Davisville, ...",39.799217,-75.049115,"[346 ARMSTRONG AVE., 6 HAMPTON CT.]"
40922,5 HAMPTON CT.,"MELBOURNE, LLC.","140 BROADWAY, 28TH FL.","NEW YORK, NY 10005","140 BROADWAY, 28TH FL., NEW YORK, NY 10005",5 HAMPTON CT. Jersey City NJ,18,1,"(5, Hampton Court, Chestnut Glen, Davisville, ...",39.799247,-75.050097,"[229 BOWERS ST., 40 MANHATTAN AVE., 132 PROSPE..."
40923,4 HAMPTON CT.,"DAVISON , JOYCE",4 HAMPTON CT. TERR.,"JERSEY CITY, N.J. 07302","4 HAMPTON CT. TERR., JERSEY CITY, N.J. 07302",4 HAMPTON CT. Jersey City NJ,1,1,"(4, Hampton Court, Chestnut Glen, Davisville, ...",39.799012,-75.049027,[4 HAMPTON CT.]
40924,3 HAMPTON CT.,"BUCK,ZACHARY C. & DIANA Y.PERALES",30 S. KINGMAN RD,"SOUTH ORANGE, NJ 07079","30 S. KINGMAN RD, SOUTH ORANGE, NJ 07079",3 HAMPTON CT. Jersey City NJ,1,1,"(3, Hampton Court, Chestnut Glen, Davisville, ...",39.799139,-75.049806,[3 HAMPTON CT.]


In [237]:
df[df.gCode.str.contains('Hudson') == True].to_csv('jersey_city.csv', index=None)

In [238]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,ownersFullMailingAddress,propertyFullAddress,numberPropertiesOwned,units,gCode,Lat,Long,propertiesOwned
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","677 LIBERTY AVE., JERSEY CITY, N.J. 07307",677 LIBERTY AVE. Jersey City NJ,1,1,"677, Liberty Avenue, Jersey City, Hudson Count...",40.759744,-74.050457,[677 LIBERTY AVE.]
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","675 LIBERTY AVE., JERSEY CITY, N.J. 07307",675 LIBERTY AVE. Jersey City NJ,1,1,"675, Liberty Avenue, Jersey City, Hudson Count...",40.759679,-74.050499,[675 LIBERTY AVE.]
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307","673 LIBERTY AVE., JERSEY CITY, NJ 07307",673 LIBERTY AVE. Jersey City NJ,1,1,"673, Liberty Avenue, Jersey City, Hudson Count...",40.759614,-74.050542,[673 LIBERTY AVE.]
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307","671 LIBERTY AVE., JERSEY CITY, NJ 07307",671 LIBERTY AVE. Jersey City NJ,1,1,"671, Liberty Avenue, Jersey City, Hudson Count...",40.759549,-74.050584,[671 LIBERTY AVE.]
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307","669 LIBERTY AVE., JERSEY CITY, N.J. 07307",669 LIBERTY AVE. Jersey City NJ,1,1,"669, Liberty Avenue, Jersey City, Hudson Count...",40.759484,-74.050626,[669 LIBERTY AVE.]
...,...,...,...,...,...,...,...,...,...,...,...,...
40921,6 HAMPTON CT.,"ST KILDA, LLC. C/O US PROPERTY FUND","200 HUDSON ST., STE 1000","JERSEY CITY, NJ 07311","200 HUDSON ST., STE 1000, JERSEY CITY, NJ 07311","6 HAMPTON CT., Jersey City, NJ",2,1,"(6, Hampton Court, Chestnut Glen, Davisville, ...",39.799217,-75.049115,"[346 ARMSTRONG AVE., 6 HAMPTON CT.]"
40922,5 HAMPTON CT.,"MELBOURNE, LLC.","140 BROADWAY, 28TH FL.","NEW YORK, NY 10005","140 BROADWAY, 28TH FL., NEW YORK, NY 10005","5 HAMPTON CT., Jersey City, NJ",18,1,"(5, Hampton Court, Chestnut Glen, Davisville, ...",39.799247,-75.050097,"[229 BOWERS ST., 40 MANHATTAN AVE., 132 PROSPE..."
40923,4 HAMPTON CT.,"DAVISON , JOYCE",4 HAMPTON CT. TERR.,"JERSEY CITY, N.J. 07302","4 HAMPTON CT. TERR., JERSEY CITY, N.J. 07302","4 HAMPTON CT., Jersey City, NJ",1,1,"(4, Hampton Court, Chestnut Glen, Davisville, ...",39.799012,-75.049027,[4 HAMPTON CT.]
40924,3 HAMPTON CT.,"BUCK,ZACHARY C. & DIANA Y.PERALES",30 S. KINGMAN RD,"SOUTH ORANGE, NJ 07079","30 S. KINGMAN RD, SOUTH ORANGE, NJ 07079","3 HAMPTON CT., Jersey City, NJ",1,1,"(3, Hampton Court, Chestnut Glen, Davisville, ...",39.799139,-75.049806,[3 HAMPTON CT.]


In [233]:
geolocator.geocode('253.5 5TH ST, JERSEY CITY, NJ')

Location(253, 5th Street, Newport, Jersey City, Hudson County, New Jersey, 07302, United States, (40.72464033333333, -74.044917, 0.0))

In [223]:
df.drop(index=df[df.gCode.str.contains('Jersey City') == False].index, inplace=True)

In [225]:
df.append(errors_df).to_pickle('jersey_city.pkl')

In [226]:
df = pd.read_pickle('jersey_city.pkl')

In [230]:
df.to_csv('jersey_city.csv', index=None)